## Chess

From a global point, we need a chess and a board. 
 * The board must show the squares, and get the different properties of squares: rows, columns, diagonals and horse movements. 
 * The pieces must have a list of available squares (possible movements and possible captures)
 * Some global properties in a game must be the castle rights, function to include checks, checkmate, available king movements, promotion. 

In [1]:
import string
import numpy as np
import copy

In [16]:
class Square():
    def __init__(self,content,coordinates):
        self.content = content
        self.coordinates = coordinates
    def __str__(self):
        return self.coordinates
    
    def __repr__(self):
        return self.__str__()
        

class Board():
    letters = sorted(string.ascii_letters[:8])
    numbers = sorted(string.digits[1:9],reverse=True)
    #Board is a list of possible occupied squares
    def __init__(self):
        #Para incluir un color 
        self.squares = np.array([[Square("/",Board.letters[i]+Board.numbers[j]) for i in range(8)] for j in range(8)])
    def __str__(self): 
        return "\n".join([" ".join([str(square.content) for square in self.squares[j]]) for j in range(8)])
    def __repr__(self):
        return self.__str__()
    #Initial set up position 
    def initial_position(self):
        left_to_right_pieces = ["R","N","B","Q","K","B","N","R"]
        for i,piece in enumerate(left_to_right_pieces): 
            self.squares[0][i].content = Piece(self,piece,"b",self.squares[0][i].coordinates)
            self.squares[7][i].content = Piece(self,piece,"w",self.squares[7][i].coordinates)
            self.squares[1][i].content = Piece(self,"P","b",self.squares[1][i].coordinates)
            self.squares[6][i].content = Piece(self,"P","w",self.squares[6][i].coordinates)
    #Add piece 
    def add_piece(self,ctype,color,position):
        self.squares[Board.numbers.index(position[1]),Board.letters.index(position[0])].content = Piece(self,ctype,color,position)
        return
    def remove_piece(self,position):
        self.squares[Board.numbers.index(position[1]),Board.letters.index(position[0])].content = "/"
        return
    def list_pieces(self):
        list_pieces = []
        for square in self.squares.flatten():
            if square.content != "/":
                list_pieces.append(square.content)
        return list_pieces
        
        
    #Auxiliary functions
    #get square
    def get_square(self,square):
        return self.squares[Board.numbers.index(square[1]),Board.letters.index(square[0])]
    #get piece given square
    def get_piece(self,square):
        return self.squares[Board.numbers.index(square[1]),Board.letters.index(square[0])].content
    #move around the board
    def go_up(self,square):
        if -1<Board.numbers.index(square[1])-1<8:
            return square[0]+Board.numbers[Board.numbers.index(square[1])-1]
        else:
            raise ValueError("Board limits!")
    def go_down(self,square):
        if -1<Board.numbers.index(square[1])+1<8:
            return square[0]+Board.numbers[Board.numbers.index(square[1])+1]
        else:
            raise ValueError("Board limits!")
    def go_left(self,square):
        if -1<Board.letters.index(square[0])-1<8:
            return Board.letters[Board.letters.index(square[0])-1]+square[1]
        else:
            raise ValueError("Board limits!")
    def go_right(self,square):
        if -1<Board.letters.index(square[0])+1<8:
            return Board.letters[Board.letters.index(square[0])+1]+square[1]
        else:
            raise ValueError("Board limits!")
    #Given type of movement,check limits, if type 
    def check_free(self,f_list,start,length = 1000):
        start_remember = start
        available_moves = []
        for f in f_list:
            start = start_remember
            try:
                start= f(start)
                counter = 1
                while ((self.get_piece(start) == "/")&(counter <= length)):
                    available_moves.append(start)
                    try:
                        start = f(start)
                        counter +=1
                    except:
                        break
            except:
                continue
        return available_moves
    
    #Given type of movements, check captures
    def check_enemies(self,f_list,start,length = 1000):
        start_remember = start
        available_captures = []
        for f in f_list:
            start = start_remember
            try:
                start= f(start)
                counter = 1
                while ((self.get_piece(start) == "/")&(counter <= length)):
                    try:
                        start = f(start)
                        counter +=1
                    except:
                        break
                if self.get_piece(start).color != self.get_piece(start_remember).color:
                    available_captures.append(start)
            except:
                continue
        return available_captures
    
    def move(self,square_start,square_end):
        self.get_square(square_end).content = self.get_square(square_start).content
        self.get_piece(square_end).position
        self.get_square(square_start).content = "/"
        return 
    

class Piece():
    #Define piece attributes : type,
    def __init__(self,board,ctype,color,position):
        self.ctype = ctype
        self.color = color
        self.position = position
        self.board = board
    #In order to have a nicer readibility
    def __repr__(self):
        return self.ctype
    #Define functions for pieces
    #1.Available squares
    def check_moves(self):
        if self.ctype == "R":
            return self.board.check_free([lambda x:self.board.go_down(x),
                                          lambda x:self.board.go_up(x),
                                          lambda x:self.board.go_left(x),
                                          lambda x:self.board.go_right(x)],self.position)
        if self.ctype == "B": 
            return self.board.check_free([lambda x:board.go_up(board.go_right(x)),
                                          lambda x:board.go_up(board.go_left(x)),
                                          lambda x:board.go_down(board.go_right(x)),
                                          lambda x:board.go_down(board.go_left(x))],self.position)
        if self.ctype == "Q":
            return self.board.check_free([lambda x:board.go_down(x),
                                          lambda x:board.go_up(x),
                                          lambda x:board.go_left(x),
                                          lambda x:board.go_right(x),
                                          lambda x:board.go_up(board.go_right(x)),
                                          lambda x:board.go_up(board.go_left(x)),
                                          lambda x:board.go_down(board.go_right(x)),
                                          lambda x:board.go_down(board.go_left(x))],self.position)
        if self.ctype == "K":
            return self.board.check_free([lambda x:board.go_down(x),
                                          lambda x:board.go_up(x),
                                          lambda x:board.go_left(x),
                                          lambda x:board.go_right(x),
                                          lambda x:board.go_up(board.go_right(x)),
                                          lambda x:board.go_up(board.go_left(x)),
                                          lambda x:board.go_down(board.go_right(x)),
                                          lambda x:board.go_down(board.go_left(x))],self.position,length = 1)
        if self.ctype == "N":
            return self.board.check_free([lambda x:board.go_up(board.go_right(board.go_right(x))),
                                          lambda x:board.go_down(board.go_right(board.go_right(x))),
                                          lambda x:board.go_up(board.go_left(board.go_left(x))),
                                          lambda x:board.go_down(board.go_left(board.go_left(x))),
                                          lambda x:board.go_left(board.go_up(board.go_up(x))),
                                          lambda x:board.go_right(board.go_up(board.go_up(x))),
                                          lambda x:board.go_left(board.go_down(board.go_down(x))),
                                          lambda x:board.go_right(board.go_down(board.go_down(x)))],
                                          self.position,length = 1)
        if self.ctype == "P":
            if (self.position[1] == "2") | (self.position[1] == "7"):
                length = 2
                if self.color == "w":
                      return self.board.check_free([lambda x:board.go_up(x)],self.position,length =length)
                if self.color == "b":
                      return self.board.check_free([lambda x:board.go_down(x)],self.position,length =length)
            else:
                length = 1
                if self.color == "w":
                      return self.board.check_free([lambda x:board.go_up(x)],self.position,length =length)
                if self.color == "b":
                      return self.board.check_free([lambda x:board.go_down(x)],self.position,length =length)
    
    def check_captures(self):
        if self.ctype == "R":
            return self.board.check_enemies([lambda x:board.go_down(x),
                                             lambda x:board.go_up(x),
                                             lambda x:board.go_left(x),
                                             lambda x:board.go_right(x)],self.position)
        if self.ctype == "B":
            return self.board.check_enemies([lambda x:board.go_up(board.go_right(x)),
                              lambda x:board.go_up(board.go_left(x)),
                              lambda x:board.go_down(board.go_right(x)),
                              lambda x:board.go_down(board.go_left(x))],self.position)
        if self.ctype == "Q":
            return self.board.check_enemies([lambda x:board.go_down(x),
                                          lambda x:board.go_up(x),
                                          lambda x:board.go_left(x),
                                          lambda x:board.go_right(x),
                                          lambda x:board.go_up(board.go_right(x)),
                                          lambda x:board.go_up(board.go_left(x)),
                                          lambda x:board.go_down(board.go_right(x)),
                                          lambda x:board.go_down(board.go_left(x))],self.position)
        if self.ctype == "K":
            return self.board.check_enemies([lambda x:board.go_down(x),
                                          lambda x:board.go_up(x),
                                          lambda x:board.go_left(x),
                                          lambda x:board.go_right(x),
                                          lambda x:board.go_up(board.go_right(x)),
                                          lambda x:board.go_up(board.go_left(x)),
                                          lambda x:board.go_down(board.go_right(x)),
                                          lambda x:board.go_down(board.go_left(x))],self.position,length = 1)
        if self.ctype == "N":
            return self.board.check_enemies([lambda x:board.go_up(board.go_right(board.go_right(x))),
                                          lambda x:board.go_down(board.go_right(board.go_right(x))),
                                          lambda x:board.go_up(board.go_left(board.go_left(x))),
                                          lambda x:board.go_down(board.go_left(board.go_left(x))),
                                          lambda x:board.go_left(board.go_up(board.go_up(x))),
                                          lambda x:board.go_right(board.go_up(board.go_up(x))),
                                          lambda x:board.go_left(board.go_down(board.go_down(x))),
                                          lambda x:board.go_right(board.go_down(board.go_down(x)))],
                                          self.position,length = 1)
        if self.ctype == "P":
            #Extra: Add en pasant
            if self.color == "w":
                return self.board.check_enemies([lambda x:board.go_right(board.go_up(x)),
                                                 lambda x:board.go_left(board.go_up(x))],self.position,length =1)
            if self.color == "b":
                return self.board.check_enemies([lambda x:board.go_right(board.go_down(x)),
                                                 lambda x:board.go_left(board.go_down(x))],self.position,length =1)


class Player():
    def __init__(self,color,board):
        self.color = color
        self.board = board
        
    def my_pieces(self):
        return [piece for piece in board.list_pieces() if piece.color == self.color]
    def enemy_pieces(self):
        return [piece for piece in board.list_pieces() if piece.color != self.color]
    
    #El jugador está en jaque
    def check(self,board):
        print([piece.position for piece in self.enemy_pieces()])
        for enemy_piece in self.enemy_pieces():
            print("_")
            print(enemy_piece.check_captures())
            print("_")
            for capture in enemy_piece.check_captures():
                print(capture)
                if board.get_piece(capture).ctype == "K":
                    return True
        return False
    #El jugador está en jaque mate
    def check_mate(self):
        if self.check(self.board):
            #El jugador está en jaque
            print("Esta en jaque")
            for piece in self.my_pieces():
                if piece.ctype == "K":
                    moves = piece.check_moves()
                    moves.extend(piece.check_captures())
                    #Move in mental board
                    im_board_fixed = copy.deepcopy(board)
                    for movement in moves:
                        im_board_temp = copy.deepcopy(im_board_fixed)
                        im_board_temp.move(piece.position,movement)
                        print(im_board_temp)
                        print("____")
                        print(Player(self.color,im_board_temp).check(im_board_temp))
                        print("____")
                        
                        #if Player(self.color,im_board_temp).check(im_board_temp) == False:
                        #    return False
                    return True
                        
            #El rey no tiene casillas disponibles
                
    #Atributos que puede tener un player: 
    #el color de las piezas con las que juega, pero igual no es necesario
    #el set de piezas

    
    
    #Definición de funciones
    # Definir una función que devuelva un True/False con respecto a si estamos en jaque
    # Lo mismo para el jaquemate 
    
    #Definir moves available para el player como una función que devuelva moves+capturas, 
    #restrinja el movimiento al rey en caso de estar en jaque y devuelva un valor de salida en caso de 
    #jaque mate. 
    #def in_check: 
        #return 
    #Lo mismo para el jaque mate
    
    
    
            

 
            
            
                
    


In [17]:
board = Board()
board.add_piece("R","w","e1")
board.add_piece("K","b","e4")
board.add_piece("P","w","f3")
player_1 = Player(board = board,color="w")
player_2 = Player(board = board,color="b")
player_2.check_mate()

['f3', 'e1']
_
['e4']
_
e4
Esta en jaque
/ / / / / / / /
/ / / / / / / /
/ / / / / / / /
/ / / / / / / /
/ / / / / / / /
/ / / / K P / /
/ / / / / / / /
/ / / / R / / /
____
['f3', 'e1']
_
['e4']
_
e4


AttributeError: 'str' object has no attribute 'ctype'

In [7]:
Player("w",im_board_temp).check(im_board_temp)

NameError: name 'im_board_temp' is not defined

In [ ]:

board_2.move("e1","e2")
board_2

In [ ]:
board

In [ ]:
board = Board()
board.initial_position()
#devuelva tu lista de piezas
mispiezas = []
for piece in board.list_pieces():
    if piece.color == "w":
        mispiezas.append(piece)
        
#Todos los movimientos
moves = []#Hay que ver como describir un movimiento
for pieza in mispiezas: 
     moves.extend(pieza.check_moves())
     moves.extend(pieza.check_captures())
     
    
moves

In [ ]:
#Estructura de juego 

#while not jaque mate: 
    #Empieza el jugador que tenga blancas
    #Se le devuelven una lista de moves con la función de available_moves de player
    #El jugador escoge un move 
    #Se ejecuta la función moves
    #Revisión de jaque y jaque mate
    #Cambia de jugador
 

In [1]:
board = Board()
#board.initial_position()
board.add_piece("R","w","e1")
board.add_piece("R","b","e4")
board.add_piece("P","w","f3")
print(board)
#board.get_piece("e4").check_moves()
print("___")

print(board)

NameError: name 'Board' is not defined

In [ ]:
board = Board()
#board.initial_position()
board.add_piece("R","w","e1")
board.add_piece("K","b","e4")
board.add_piece("P","w","f3")
print(board)


In [ ]:
# Posible definición de check 
for piece in board.list_pieces():
    possible_captures = piece.check_captures() 
    if possible_captures != []:
        for capture in possible_captures:
            if board.get_piece(capture).ctype == "K":
                check = True

In [ ]:
#if jaque : 
    #jugador escoge movimiento:
    #if movimiento -> jaque: 
        #"Movimiento no valido"
    